In [ ]:
import sys; sys.path.append("..") 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.utility import loadfile, savefile, exists
from src.dataprocessing import *
from src import rnn

## load data

In [ ]:
# Define variables
input_var   = ["FT-3061-2", "FT-3061-3", "FT-3061-4", "FT-3062-1"]
output_var  = ["TT-3061-3", "TT-3061-5", "LT-3061-2"]
process_var = input_var + output_var

In [ ]:
min_len = 100
continuous_path = './data/3_continuous'

cts_list = []
i = 1
while exists(continuous_path, f"cts_{min_len}/dataset {min_len}_{i}", 'csv'):
    cts_df = loadfile(continuous_path, f"cts_{min_len}/dataset {min_len}_{i}", 'csv')
    cts_list.append(cts_df)
    i += 1

In [5]:
target_list = cts_list

# history size and future size
history_list = [10, 20, 30, 40]
future_list = [10, 20, 30]
step = 1

# variable selection
history_var = process_var
future_var = output_var

history_num = len(history_var)
future_num = len(future_var)

# supervised attention factor
delta = 1
att_type = 'linear'
        
# test data split        
test_size = 0.2
test_num = -1

# model structure
num_layers = 1
num_neurons = 50
dense_layers = 1
dense_neurons = 50
model_type = 'datt_seq2seq_gru'

iteration_list = [5, 6]
for iteration in iteration_list:
    for history_size in history_list:
        for future_size in future_list:
            print(f"{iteration}th iteration")
            print(f"history size: {history_size}")
            print(f"future size: {future_size}")
            history_series = []
            future_series = []

            # data to series
            for i in range(len(target_list)):
                history, future = data2series(target_list[i], history_size, history_var, future_size, future_var,
                                            step, start_idx=0, end_idx=None)
                if not i:
                    history_series = history
                    future_series = future
                else:
                    history_series = np.concatenate([history_series, history], axis=0)
                    future_series = np.concatenate([future_series, future], axis=0)
            
            factor = rnn.super_attention(delta, future_size, future_num, att_type)
            # Dual-attention Seq2Seq model
            DATT_seq2seq_GRU = rnn.RNN(history_series, history_var, future_series, future_var)
            # TT split
            DATT_seq2seq_GRU.train_test(test_size=test_size, test_num=test_num)
            # TV split
            valid_size = DATT_seq2seq_GRU.history_test.shape[0]/DATT_seq2seq_GRU.history_train.shape[0]
            DATT_seq2seq_GRU.train_valid(valid_size=valid_size)
            # scaling
            DATT_seq2seq_GRU.scaling()
            # modeling
            DATT_seq2seq_GRU.build_model(num_layers=num_layers, num_neurons=num_neurons, dense_layers=dense_layers, dense_neurons=dense_neurons, model_type=model_type, factor=factor)
            # training
            model_num = iteration
            model_name = f"{history_size}_{future_size}_{num_layers}_{num_neurons}_{dense_layers}_{dense_neurons}_{model_type}_{model_num}"
            if not exists("./model", model_name, 'model'):
                DATT_seq2seq_GRU.train()
                DATT_seq2seq_GRU.save_model(f"./model/{model_name}", 'weights')
                
            else:
                DATT_seq2seq_GRU.model.load_weights(f"./model/{model_name}.h5")
            # test
            test_result = DATT_seq2seq_GRU.test()
            if not exists('./result', model_name):
                savefile(test_result, './result', model_name)
            print("\n")
    print('\n\n')
print('end opitmization')

5th iteration
history size: 10
future size: 10
model is loaded from: ./model/10_10_1_50_1_50_datt_seq2seq_gru_5


,TT-3061-3,TT-3061-5,LT-3061-2,mean,TT-3061-3,TT-3061-5,LT-3061-2,mean
index,R2,R2,R2,R2,nRMSE,nRMSE,nRMSE,nRMSE
0,0.95191,0.940408,0.968614,0.953644,3.560584,4.268898,2.762227,3.53057
1,0.890604,0.893705,0.927472,0.903927,5.335878,5.702014,4.197904,5.078599
2,0.803316,0.846414,0.888765,0.846165,7.057305,6.854641,5.197486,6.36981
3,0.705844,0.807029,0.856355,0.789743,8.516202,7.684496,5.904804,7.3685
4,0.606356,0.781837,0.834288,0.740827,9.755366,8.171844,6.340142,8.089117
5,0.508723,0.762108,0.816329,0.69572,10.793311,8.534796,6.673193,8.6671
6,0.416878,0.743919,0.802251,0.654349,11.6476,8.855826,6.922954,9.142127
7,0.33084,0.726027,0.793323,0.61673,12.447698,9.160305,7.076812,9.561605
8,0.255451,0.709155,0.786049,0.583552,13.145387,9.438589,7.199726,9.927901




5th iteration
history size: 10
future size: 20
model is loaded from: ./model/10_20_1_50_1_50_datt_seq2seq_gru_5


,TT-3061-3,TT-3061-5,LT-3061-2,mean,TT-3061-3,TT-3061-5,LT-3061-2,mean
index,R2,R2,R2,R2,nRMSE,nRMSE,nRMSE,nRMSE
0,0.907338,0.9236,0.942273,0.924404,6.0125,4.829025,3.738263,4.859929
1,0.831534,0.876888,0.906706,0.871709,7.959419,6.131919,4.75183,6.281056
2,0.749568,0.83968,0.867606,0.818951,9.280731,7.000062,5.659884,7.313559
3,0.664425,0.810446,0.822773,0.765882,10.384433,7.614201,6.546914,8.18185
4,0.576804,0.792378,0.778699,0.71596,11.475779,7.972397,7.313586,8.920587
5,0.489491,0.776837,0.750726,0.672352,12.562948,8.269437,7.759246,9.530544
6,0.411084,0.760598,0.734248,0.63531,13.229992,8.567639,8.009402,9.935678
7,0.344047,0.741391,0.717437,0.600958,13.69563,8.907545,8.256984,10.286719
8,0.286228,0.719188,0.696239,0.567218,14.074536,9.285111,8.559255,10.639634




5th iteration
history size: 10
future size: 30
model is loaded from: ./model/10_30_1_50_1_50_datt_seq2seq_gru_5


...



5th iteration
history size: 30
future size: 10
Epoch 1/10000
572/572 - 8s - loss: 0.2375 - val_loss: 0.1532 - 8s/epoch - 14ms/step
Epoch 2/10000
572/572 - 5s - loss: 0.1503 - val_loss: 0.1407 - 5s/epoch - 9ms/step
Epoch 3/10000
572/572 - 5s - loss: 0.1369 - val_loss: 0.1299 - 5s/epoch - 9ms/step
Epoch 4/10000
572/572 - 5s - loss: 0.1289 - val_loss: 0.1216 - 5s/epoch - 9ms/step
Epoch 5/10000
572/572 - 5s - loss: 0.1210 - val_loss: 0.1130 - 5s/epoch - 8ms/step
Epoch 6/10000
572/572 - 5s - loss: 0.1128 - val_loss: 0.1081 - 5s/epoch - 9ms/step
Epoch 7/10000
572/572 - 5s - loss: 0.1052 - val_loss: 0.1001 - 5s/epoch - 9ms/step
Epoch 8/10000
572/572 - 5s - loss: 0.1013 - val_loss: 0.0962 - 5s/epoch - 9ms/step
Epoch 9/10000
572/572 - 5s - loss: 0.0965 - val_loss: 0.0939 - 5s/epoch - 9ms/step
Epoch 10/10000
572/572 - 5s - loss: 0.0938 - val_loss: 0.0949 - 5s/epoch - 9ms/step
Epoch 11/10000
572/572 - 5s - loss: 0.0893 - val_loss: 0.0938 - 5s/epoch - 9ms/step
Epoch 12/10000
572/572 - 5s - loss:

INFO:tensorflow:Assets written to: ./model/30_10_1_50_1_50_datt_seq2seq_gru_5\assets


INFO:tensorflow:Assets written to: ./model/30_10_1_50_1_50_datt_seq2seq_gru_5\assets


,TT-3061-3,TT-3061-5,LT-3061-2,mean,TT-3061-3,TT-3061-5,LT-3061-2,mean
index,R2,R2,R2,R2,nRMSE,nRMSE,nRMSE,nRMSE
0,0.924327,0.939238,0.964059,0.942541,4.464704,4.329941,2.927633,3.907426
1,0.838982,0.890529,0.913413,0.880975,6.469446,5.812566,4.543327,5.608446
2,0.73022,0.845909,0.856926,0.811019,8.257909,6.896793,5.839749,6.99815
3,0.618659,0.810308,0.822115,0.750361,9.685695,7.653146,6.511593,7.950144
4,0.512948,0.779868,0.8093,0.700705,10.837513,8.245457,6.741878,8.608283
5,0.418388,0.754449,0.807754,0.660197,11.728326,8.70983,6.769212,9.069123
6,0.336055,0.734385,0.806868,0.625769,12.412526,9.059331,6.785116,9.418991
7,0.265017,0.719828,0.80139,0.595412,13.030973,9.304513,6.881273,9.73892
8,0.203457,0.708651,0.786828,0.566312,13.584765,9.488566,7.129562,10.067631


csv file is saved to: ./result/30_10_1_50_1_50_datt_seq2seq_gru_5.csv


5th iteration
history size: 30
future size: 20
Epoch 1/10000
568/568 - 9s - loss: 0.3089 - val_loss: 0.2612 - 9s/epoch - 15ms/step
Epoch 2/10000
568/568 - 5s - loss: 0.2163 - val_loss: 0.2358 - 5s/epoch - 8ms/step
Epoch 3/10000
568/568 - 5s - loss: 0.2018 - val_loss: 0.2248 - 5s/epoch - 8ms/step
Epoch 4/10000
568/568 - 5s - loss: 0.1914 - val_loss: 0.2183 - 5s/epoch - 8ms/step
Epoch 5/10000
568/568 - 5s - loss: 0.1825 - val_loss: 0.2061 - 5s/epoch - 8ms/step
Epoch 6/10000
568/568 - 5s - loss: 0.1752 - val_loss: 0.1949 - 5s/epoch - 8ms/step
Epoch 7/10000
568/568 - 5s - loss: 0.1664 - val_loss: 0.1907 - 5s/epoch - 8ms/step
Epoch 8/10000
568/568 - 5s - loss: 0.1592 - val_loss: 0.1810 - 5s/epoch - 8ms/step
Epoch 9/10000
568/568 - 5s - loss: 0.1514 - val_loss: 0.1771 - 5s/epoch - 8ms/step
Epoch 10/10000
568/568 - 5s - loss: 0.1441 - val_loss: 0.1663 - 5s/epoch - 8ms/step
Epoch 11/10000
568/568 - 5s - loss: 0.1409 - val_